In [ ]:
# pip install openai==0.28.0

In [7]:
import os
import json
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import openai
import torch

# CLIP 및 GPT 모델 설정
repo = "Bingsu/clip-vit-large-patch14-ko"
clip_model = CLIPModel.from_pretrained(repo)
clip_processor = CLIPProcessor.from_pretrained(repo)

# 폴더 경로 설정
SEED_TASK_FOLDER = "/content/drive/MyDrive/[빅컴]프로젝트/vlm-main/Data/KFashion_Image/Final_Data_2"
SAVE_FOLDER = "/content/drive/MyDrive/[빅컴]프로젝트/vlm-main/Data/KFashion_Image/Generated2"
os.makedirs(SAVE_FOLDER, exist_ok=True)

# 다양한 속성 정의
style = ['레트로', '로맨틱', '리조트', '매니시', '모던', '밀리터리', '섹시', '소피스트케이티드', '스트리트', '스포티', '아방가르드', '오리엔탈', '웨스턴', '젠더리스',
         '컨트리', '클래식', '키치', '톰보이', '펑크', '페미닌', '프레피', '히피', '힙합']
substyle = ['레트로', '로맨틱', '리조트', '매니시', '모던', '밀리터리', '소피스트케이티드', '스트리트', '스포티', '오리엔탈', '웨스턴', '젠더리스', '컨트리', '클래식', '톰보이', '페미닌', '히피']
outers_lengh = ['노말', '롱', '맥시', '크롭', '하프']
outers_category = ['가디건', '베스트', '재킷', '점퍼', '짚업', '코트', '패딩']
pants_lengh = ['니렝스', '맥시', '미니', '미디', '발목']
pants_category = ['래깅스', '스커트', '조거팬츠', '청바지', '팬츠']
dress_lengh = ['니렝스', '맥시', '미니', '미디', '발목']
dress_category = ['드레스', '점프수트']
top_lengh = ['노멀', '롱', '크롭']
top_category = ['니트웨어', '블라우스', '셔츠', '탑', '티셔츠', '후드티']
color = ['골드', '그레이', '그린', '네이비', '라벤더', '레드', '민트', '베이지', '브라운', '블랙', '블루', '스카이블루', '실버', '옐로우', '오렌지', '와인', '카키', '퍼플', '핑크', '화이트']
subcolor = ['골드', '그레이', '그린', '네이비', '라벤더', '레드', '민트', '베이지', '브라운', '블랙', '블루', '스카이블루', '실버', '옐로우', '오렌지', '와인', '카키', '퍼플', '핑크', '화이트']
pattern = ['없음','무지','스프라이프','격자','도트','꽃','호피','얼룩말','헤링본','지그재그','사각형','삼각형','원','체크','추상적','타이다이','쉐브론','아가일','레이스','물결','다마스크',
           '셔링','트위드','캐릭터 그림','영어 프린팅','한글 프린팅','밀리터리','랩', '단추', '배색', '스모크', '로고', '하트','꽈배기','털']
neck_line = ['라운드넥', '보트넥', '브이넥', '스위트하트', '스퀘어넥', '오프숄더', '원숄더', '유넥', '터틀넥']
##categories = ["탑",'가디건', '베스트', '재킷', '점퍼', '짚업', '코트', '패딩','래깅스', '스커트', '조거팬츠', '청바지', '팬츠','드레스', '점프수트']
materials = ['가죽', '네오프렌', '니트', '데님', '레이스', '린넨', '무스탕', '비닐/PVC', '스판덱스', '시폰', '실크', '우븐', '울/캐시미어', '저지', '코듀로이', '트위드', '패딩', '퍼', '플리스']
fits = ['노멀', '루즈', '벨보텀', '스키니', '오버사이즈', '와이드', '타이트']
prints = ['그래픽', '깅엄', '도트', '레터링', '무지', '믹스', '스트라이프', '지그재그', '체크', '카무플라쥬', '페이즐리', '플로럴', '하운즈 투스', '호피']
details = ['니트꽈베기', '단추', '더블브레스티드', '드롭숄더', '드롭웨이스트', '디스트로이드', '띠', '러플', '레이스', '롤업', '리본', '버클', '비대칭', '비즈', '셔링',
           '스터드', '스트링', '스티치', '슬릿', '싱글브레스티드', '자수', '지퍼', '컷아웃', '퀄팅', '태슬', '패치워크', '퍼트리밍', '퍼프', '페플럼', '포켓', '폼폼', '프린지', '프릴', '플레어', '플리츠']
sleeves = ['7부소매', '긴팔', '민소매', '반팔', '없음', '캡']
occasions = ["친구와의 브런치","가벼운 산책","쇼핑 데이","카페에서 독서","도서관 방문","공원 피크닉","주말 영화관 나들이","반려동물 산책","도심 속 자전거 타기","마트 장보기","카페 방문",
             "직장 면접","회사 출근","클라이언트 미팅","공식 발표회","업무 워크숍", "사내 프레젠테이션","네트워킹 행사","고급 레스토랑 점심",
             "결혼식 하객룩","생일 파티","연말 파티","주말 저녁 술자리","칵테일 파티","축하 이벤트","미술관 개관식",
             "해변 휴가","캠핑","해외여행 도시 투어","산악 하이킹","봄꽃 축제","계곡 피크닉","온천 여행","비행기 기내 복장","휴양지",
             "봄 벚꽃놀이","여름 해변가","가을 단풍 구경","겨울 크리스마스 마켓","장마철 외출","무더운 여름 낮","쌀쌀한 가을 저녁","한파 속 출근길",
             "저녁 데이트","야경이 멋진 레스토랑","놀이공원 데이트","감성 카페 데이트","겨울 아이스링크 데이트","드라이브 데이트","별빛이 보이는 루프탑","피크닉 데이트",
             "동네 산책", "요가 수업","헬스장 운동","등산","자전거 타기","러닝","필라테스 수업","배드민턴 경기","집","할로윈","데일리"]
seasons = ["봄", "여름", "가을", "겨울", "사계절"]
shoes = ['없음','운동화','단화','블로퍼','워커','구두','하이힐','힐','로퍼', '부츠','롱부츠','플랫슈즈','슬리퍼','샌달','첼시 부츠','앵클 부츠','러닝화']
items = ['없음','토트백','백팩','에코백','핸드백','라탄백','종이백','볼캡','플라스틱 핸드백','핸드폰','꽃','파우치','커피','밀짚모자','손수건','팔찌','모자', '책']

# CLIP을 사용하여 속성 추출 (상위 1개 후보 반환)
def extract_top_matches(image, prompts, top_k=1, max_length=70):
    inputs = clip_processor(text=prompts, images=image, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = clip_model(**inputs)
    probs = outputs.logits_per_image.softmax(dim=1)
    top_indices = torch.topk(probs, k=top_k, dim=1).indices[0]
    return [prompts[i] for i in top_indices]

# 주요 속성 추출 함수
def extract_image_attributes(image_path):
    image = Image.open(image_path)
    style_candidates = extract_top_matches(image, style)
    substyle_candidates = extract_top_matches(image, substyle)
    outer_length_candidates = extract_top_matches(image, outers_lengh)
    outer_category_candidates = extract_top_matches(image, outers_category)
    pant_length_candidates = extract_top_matches(image, pants_lengh)
    pant_category_candidates = extract_top_matches(image, pants_category)
    dress_length_candidates = extract_top_matches(image, dress_lengh)
    dress_category_candidates = extract_top_matches(image, dress_category)
    top_length_candidates = extract_top_matches(image, top_lengh)
    top_category_candidates = extract_top_matches(image, top_category)
    color_candidates = extract_top_matches(image, color)
    subcolor_candidates = extract_top_matches(image, subcolor)
    pattern_candidates = extract_top_matches(image, pattern)
    neck_line_candidates = extract_top_matches(image, neck_line)
    ##category_candidates = extract_top_matches(image, categories)
    material_candidates = extract_top_matches(image, materials)
    fit_candidates = extract_top_matches(image, fits)
    print_candidates = extract_top_matches(image, prints)
    detail_candidates = extract_top_matches(image, details)
    sleeve_candidates = extract_top_matches(image, sleeves)
    occasion_candidates = extract_top_matches(image, occasions)
    season_candidates = extract_top_matches(image, seasons)
    shoe_candidates = extract_top_matches(image, shoes)
    item_candidates = extract_top_matches(image, items)

    return {
        "style": style_candidates,
        "substyle": substyle_candidates,
        "outer_length": outer_length_candidates,
        "outer_category": outer_category_candidates,
        "pant_length": pant_length_candidates,
        "pant_category": pant_category_candidates,
        "dress_length": dress_length_candidates,
        "dress_category": dress_category_candidates,
        "top_length": top_length_candidates,
        "top_category": top_category_candidates,
        "color": color_candidates,
        "subcolor": subcolor_candidates,
        "pattern": pattern_candidates,
        "neck_line": neck_line_candidates,
        ##"category": category_candidates,
        "material": material_candidates,
        "fit": fit_candidates,
        "print": print_candidates,
        "detail": detail_candidates,
        "sleeve": sleeve_candidates,
        "occasion": occasion_candidates,
        "season": season_candidates,
        "shoe": shoe_candidates,
        "item": item_candidates,
    }



# Input caption과 Add_Info 생성 함수
def generate_instruction_content(attributes):
    input_caption = (
        f"스타일 : {attributes['style']}, "
        f"서브스타일 : {attributes['substyle']}, "
        f"아우터 : {attributes['outer_category']}, "
        f"하의 : {{'기장': '{attributes['pant_length']}', '색상': '{attributes['color']}', '서브색상' : '{attributes['color']}', '카테고리': '{attributes['pant_category']}', "
        f"'소재': ['{attributes['material']}'], '프린트': ['{attributes['pattern']}'], '핏': '{attributes['fit']}'}}, "
        f"원피스 : {{'기장': '{attributes['dress_length']}', '색상': '{attributes['color']}', '서브색상' : '{attributes['color']}', '카테고리': '{attributes['dress_category']}, '디테일': '{attributes['detail']}}}, "
        f"상의 : {{'기장': '{attributes['top_length']}', '색상': '{attributes['color']}', '카테고리': '{attributes['top_category']}', "
        f"'소매기장': '{attributes['sleeve']}', '소재': ['{attributes['material']}'], '프린트': ['{attributes['pattern']}'], "
        f"'넥라인': '{attributes['neck_line']}', '핏': '{attributes['fit']}'}}"
    )

    add_info = (
        f"어울리는 상황: {attributes['occasion']}, "
        f"어울리는 계절: {attributes['season']}, "
        #f"패턴: {attributes['pattern']}, "
        f"신발: {attributes['shoe']}, "
        #f"소품: {attributes['item']} "
    )

    return input_caption, add_info

# GPT로 텍스트 생성
# def generate_gpt_caption(attributes):
#     prompt = (
#         f"다음 속성을 기반으로 한국 20-30대 여성을 위한 패션 스타일 설명을 작성해 주세요:\n\n"
#         f"스타일 : {', '.join(attributes['style'])}\n"
#         f"서브스타일 : {', '.join(attributes['substyle'])}\n"
#         f"아우터 : {', '.join(attributes['outer_category'])}\n"
#         f"하의 : {', '.join(attributes['pant_category'])}\n"
#         f"원피스 : {', '.join(attributes['dress_category'])}\n"
#         f"상의 : {', '.join(attributes['top_category'])}\n"
#         f"소재 : {', '.join(attributes['material'])}\n"
#         f"색상 : {', '.join(attributes['color'])}\n"
#         f"서브색상 : {', '.join(attributes['subcolor'])}\n"
#         f"프린트 : {', '.join(attributes['pattern'])}\n"
#         f"디테일 : {', '.join(attributes['detail'])}\n"
#         f"핏 : {', '.join(attributes['fit'])}\n"
#         f"소매기장 : {', '.join(attributes['sleeve'])}\n"
#         f"넥라인 : {', '.join(attributes['neck_line'])}\n"
#     )

#     response = openai.ChatCompletion.create(
#         model="gpt-4",
#         messages=[
#             {"role": "system", "content": "당신은 패션 전문가입니다."},
#             {"role": "user", "content": prompt}
#         ],
#         temperature=0.7,
#         max_tokens=150
#     )
#     return response['choices'][0]['message']['content'].strip()

# 이미지 파일에 대한 지시 사항 생성 및 저장
for file_name in os.listdir(SEED_TASK_FOLDER):
    if file_name.endswith(".jpg") or file_name.endswith(".jpeg") or file_name.endswith(".png"):
        image_path = os.path.join(SEED_TASK_FOLDER, file_name)
        json_file_name = file_name.rsplit(".", 1)[0] + ".json"
        json_path = os.path.join(SEED_TASK_FOLDER, json_file_name)

        # JSON 파일이 없는 JPG 파일만 처리
        if not os.path.exists(json_path):
            try:
                # 이미지 속성 추출
                attributes = extract_image_attributes(image_path)

                # Input caption과 Add_Info 생성
                input_caption, add_info = generate_instruction_content(attributes)

                # GPT Caption 생성
                #gpt_caption = generate_gpt_caption(attributes)

                # 생성된 지시 사항을 저장
                generated_instruction = {
                    "Prompt": {
                        "persona": "당신은 한국 20-30대 여성을 타겟으로 하는 패션 디자이너입니다.",
                        "task_description": "Input에 주어진 스타일을 기반으로 여성이 옷을 입은 모습을 생성해주세요.",
                        "constraint": "얼굴을 제외하고 여성의 전신이 나오도록 옷 입은 모습 이미지를 생성해주세요, 그림 형식이 아니라 사진 형식이어야 합니다."
                    },
                    "Input": {
                        "caption": input_caption
                    },
                    "Add_Info": add_info,
                    #"GPT_Caption": gpt_caption
                }

                # 저장 파일 이름 설정
                save_file_name = f"{file_name.rsplit('.', 1)[0]}.json"
                save_path = os.path.join(SAVE_FOLDER, save_file_name)

                # JSON 파일로 저장
                with open(save_path, "w", encoding="utf-8") as f:
                    json.dump(generated_instruction, f, ensure_ascii=False, indent=4)

                print(f"{save_file_name}이 {SAVE_FOLDER}에 저장되었습니다.")

            except Exception as e:
                print(f"{file_name} 처리 중 오류 발생: {e}")


In [ ]:
# 속성 확인
print("Extracted Attributes:", attributes)

# Add_Info 생성 디버깅
print("Add_Info 생성 값:")
print("어울리는 상황:", attributes['occasion'])
print("소품:", attributes['item'])  # 올바르게 반영되는지 확인